In [1]:
# 1.04.2024
# обновлена схема, построены графики для пилотного сигнала

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

На картинке изображена структурная схема модели, которую планируется реализовать.

- Желтым цветом отмечены блоки, функции для которых на данный момент прописаны.

- Синим цветом отмечены блоки, над которыми либо ведется работа, либо они просто пропущены (тк не написан другой блок, без которого этот не может обойтись), но будут в финальной модели.

- Красным цветом отмечены блоки, которые делать не планируется.

![Структурная схема модели](model_structure.png)

Каждый блок этой диаграммы (или набор блоков) реализован как отдельная функция, преобразующая входной в нее массив.

Основной управляющий файл `ofdm_modem.m` сохраняет данные с выхода каждого блока в txt файл. Поэтому можно смотреть как преобразуется сигнал от блока к блоку.

На вход он принимает следующие параметры - порядок модулятора `M` (2,4,8,16,64...); длину фрейма `fr_len` (степень двойки); соотношение сигнал/шум на приемнике в децибелах `SNR_dB`; длину циклического префикса `cp_length`

In [1]:
M = 16; # e.g. 2, 4, 8 -> PSK; 16, 64... -> QAM
fr_len = 64; # the length of OFDM frame
SNR_dB = 20; # [dBW] the signal power is normalized to 1 W
cp_length = fr_len/2; # the size of cyclic prefix

# Передатчик

Включает все бллоки до передачи сигнала в канал

## Блок Bits stream

Генерирует случайную последовательность чисел от 0 до `M-1` длины такой же как длина фрейма. Используется функция `randi()`

## Блок Modulator

# Канал

# Приемник